In [ ]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [ ]:
def get_data_from_dir(dir):
    tweet_file = [(dir + file, file)  for file in os.listdir(dir) if 'zip' in file]
    return tweet_file

In [ ]:
tweet_list = get_data_from_dir('../AllData/dataset/')
tweet_list

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('model_v2')

for file, name in tweet_list:
    data = pd.read_csv(file, compression='zip', lineterminator='\n')
    data['Mean'] = model.predict(data['Text'].to_numpy())
    data = data[data['Datetime'] != 'Datetime' ]
    data = data[['Datetime', 'Mean']]
    data['Datetime'] = data['Datetime'].apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))

    data_want = data.groupby(['Datetime']).mean(numeric_only = True)

    data_want['SD'] = data.groupby(['Datetime']).std(numeric_only = True)['Mean']

    data_want['Datetime'] = data_want.index

    data_want.to_csv(f'../AllData/pred_tweet/{name}', compression={'method': 'zip', 'archive_name': f'{name[:-4]}'}, index = None)
